# 特殊方法
python中特殊方法用两个下划线结尾，如\__len__ 和\__getitem__，示例如下：

In [62]:
import collections
# 具名元组可用于构建仅含有属性的类，需指定类名和属性名list
Card = collections.namedtuple('Card', ['rank','suit'])
class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + ['J','Q','K','A']
    suits = ['spades','diamonds','clubs','hearts']
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, index):
        return self._cards[index]
    
deck = FrenchDeck()

In [7]:
deck[0]

Card(rank='2', suit='spades')

In [8]:
len(deck)

52

1.支持python特殊方法的deck就可以使用python本身提供的方式去完成相关功能，例如由于实现了\__getitem__，deck对象支持python内置的从序列中随机抽取元素的函数random.choice。

2.由于\__getitem__实际上将[]操作交给了self._cards，当使用切片时（index是切片对象）deck类自动支持切片索引。

3.由于支持\__getitem__，deck可迭代，因此可用python自带的排序方法。

4.\__contains__方法用于in运算法，如果未实现该方法则python会调用\__getitem__做一遍迭代搜索。

### 由此可见，实现特殊方法，自定义数据类型可以表现得像python内置类型一样，从而实现更具表达力或python风格的代码。

In [56]:
import random
random.choice(deck)

Card(rank='8', suit='spades')

In [57]:
deck[0:5]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades'),
 Card(rank='5', suit='spades'),
 Card(rank='6', suit='spades')]

In [67]:
suit_vals = dict(spades=3, hearts=2, diamonds=1, clubs=0)
def spades_high(card):
    rank_val = FrenchDeck.ranks.index(card.rank)
    suit_val = suit_vals.get(card.suit)
    return rank_val * len(suit_vals) + suit_val

for card in sorted(deck, key=spades_high):
    print(card)

Card(rank='2', suit='clubs')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='hearts')
Card(rank='2', suit='spades')
Card(rank='3', suit='clubs')
Card(rank='3', suit='diamonds')
Card(rank='3', suit='hearts')
Card(rank='3', suit='spades')
Card(rank='4', suit='clubs')
Card(rank='4', suit='diamonds')
Card(rank='4', suit='hearts')
Card(rank='4', suit='spades')
Card(rank='5', suit='clubs')
Card(rank='5', suit='diamonds')
Card(rank='5', suit='hearts')
Card(rank='5', suit='spades')
Card(rank='6', suit='clubs')
Card(rank='6', suit='diamonds')
Card(rank='6', suit='hearts')
Card(rank='6', suit='spades')
Card(rank='7', suit='clubs')
Card(rank='7', suit='diamonds')
Card(rank='7', suit='hearts')
Card(rank='7', suit='spades')
Card(rank='8', suit='clubs')
Card(rank='8', suit='diamonds')
Card(rank='8', suit='hearts')
Card(rank='8', suit='spades')
Card(rank='9', suit='clubs')
Card(rank='9', suit='diamonds')
Card(rank='9', suit='hearts')
Card(rank='9', suit='spades')
Card(rank='10', suit='clubs')
Ca

In [63]:
Card('Q', 'hearts') in deck

True

## 特殊方法的使用原则
特殊方法是为了被python解释器识别调用的不应该自行调用，另外python解释器会对内置的类型如list、str、bytearray做优化，如调用len()时，CPython会访问内置类型对象在内存中长度可变的C语言结构体PyVarObject，直接读取ob_size属性。

特殊方法的调用通常是隐式的（相对于len），如for i in x:，该语句会调用x.\__iter__()。

## 运算符重载模拟数值运算

In [79]:
class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    def __repr__(self):
        """
        1.对象的字符串表示，否则可能是<Vector object at 0x10e100070>,对应的占位符为%r。
        这里对self.x, self.y同样使用%r去调用其对应的__repr__方法。
        2.__repr__应尽量反应如何用代码创建出这个被打印的对象。
        3.__str__在str()时调用，可以仅仅实现__repr__，当__str__不存在时python解释器会调用__repr__方法。
        """
        return 'Vector(%r, %r)' % (self.x, self.y)
    
    def __abs__(self):
        """
        通过abs()函数使用。
        """
        return hypot(self.x, self.y)
    
    def __bool__(self):
        """
        bool(),若类没有实现__bool__则调用__len__，当返回0时bool返回False，否则为True。
        or作为短路运算符返回x（x为真时）或y。
        """
        return bool(self.x or self.y)
    
    def __add__(self, other):
        """
        +重载。
        """
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)
    
    def __mul__(self, scaler):
        """
        *符号重载，但只能做被乘数，因为scaler不是Vecter对象。
        """
        return Vector(self.x * scaler, self.y * scaler)

In [85]:
v1 = Vector(2, 4)
v2 = Vector(2, 4)
v2 + v1

Vector(4, 8)